<a href="https://colab.research.google.com/github/BingHung/AI/blob/master/%5B03032019_1%5D_RT_TEST_BATCH100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse

import numpy as np
import tensorflow as tf
import os.path
import sys
import re
import hashlib
import pandas as pd
from datetime import datetime
import csv 

def load_graph(model_file):                                  #讀取已經訓練好的模型 是.pb檔
  graph = tf.Graph()
  graph_def = tf.GraphDef()

  with open(model_file, "rb") as f:
    graph_def.ParseFromString(f.read())
  with graph.as_default():
    tf.import_graph_def(graph_def)

  return graph


def read_tensor_from_image_file(file_name,                   #修改欲預測的圖片尺寸 變成299*299
                                input_height=299,
                                input_width=299,
                                input_mean=0,
                                input_std=255):

  input_name = "file_reader"
  output_name = "normalized"
  file_reader = tf.read_file(file_name, input_name)
  if file_name.endswith(".png"):
    image_reader = tf.image.decode_png(
        file_reader, channels=3, name="png_reader")
  elif file_name.endswith(".gif"):
        image_reader = tf.squeeze(
        tf.image.decode_gif(file_reader, name="gif_reader"))
  elif file_name.endswith(".bmp"): 
    image_reader = tf.image.decode_bmp(file_reader, name="bmp_reader")
  else:
    image_reader = tf.image.decode_jpeg(
        file_reader, channels=3, name="jpeg_reader")
  float_caster = tf.cast(image_reader, tf.float32)
  dims_expander = tf.expand_dims(float_caster, 0)
  resized = tf.image.resize_bilinear(dims_expander, [input_height, input_width])
  normalized = tf.divide(tf.subtract(resized, [input_mean]), [input_std])
  with tf.Session() as sess:
      a = sess.run(float_caster)
      result = sess.run(normalized)
  # sess = tf.Session()
  
  return result


def load_labels(label_file):                               #讀取用來分類的label檔案

  label = []
  proto_as_ascii_lines = tf.gfile.GFile(label_file).readlines()
  for l in proto_as_ascii_lines:
    label.append(l.rstrip())

  return label

def func(listTemp, n):
    for i in range(0, len(listTemp), n):
        yield listTemp[i:i + n]


if __name__ == "__main__":
  file_name = "tensorflow/examples/label_image/data/grace_hopper.jpg"
  model_file = \
    "tensorflow/examples/label_image/data/inception_v3_2016_08_28_frozen.pb"
  label_file = "tensorflow/examples/label_image/data/imagenet_slim_labels.txt"
  input_height = 299
  input_width = 299
  input_mean = 0
  input_std = 255
  input_layer = "input"
  output_layer = "InceptionV3/Predictions/Reshape_1"

  parser = argparse.ArgumentParser()                           #設定可以輸入的變數,例如選擇哪個模型 選擇要預測的圖片
  parser.add_argument("--image", help="image to be processed")
  parser.add_argument("--graph", help="graph/model to be executed")
  parser.add_argument("--labels", help="name of file containing labels")
  parser.add_argument("--input_height", type=int, help="input height")
  parser.add_argument("--input_width", type=int, help="input width")
  parser.add_argument("--input_mean", type=int, help="input mean")
  parser.add_argument("--input_std", type=int, help="input std")
  parser.add_argument("--input_layer", help="name of input layer")
  parser.add_argument("--output_layer", help="name of output layer")
  args = parser.parse_args()

  if args.graph:
    model_file = args.graph
  if args.image:
    file_name = args.image
  if args.labels:
    label_file = args.labels
  if args.input_height:
    input_height = args.input_height
  if args.input_width:
    input_width = args.input_width
  if args.input_mean:
    input_mean = args.input_mean
  if args.input_std:
    input_std = args.input_std
  if args.input_layer:
    input_layer = args.input_layer
  if args.output_layer:
    output_layer = args.output_layer

  #========================================================
  # MAX_NUM_IMAGES_PER_CLASS = 2 ** 27 - 1  # ~134M

  image_dir = file_name # "test_loop"
  files_list = []
  file_glob = os.path.join(image_dir, '*.jpg')
  files_list.extend(tf.gfile.Glob(file_glob)) #展開所有檔案並存入 files_list

  x = []
  for file_list in func(files_list, 100):
   # print('====\n'*5,len(file_list),'====\n'*5)
        
    iCount =0
    ans = pd.DataFrame() #轉csv檔使用
    ans2=[]
    ans3=[]
    ans4=[]
    ok_labels =["ok", "OK", "Ok", "normal", "NORMAL", "Normal"]
    
    graph = load_graph(model_file)
    test_images = list()
    
    for file_name in file_list: 
      t = read_tensor_from_image_file(
        file_name,
        input_height=input_height,
        input_width=input_width,
        input_mean=input_mean,
        input_std=input_std)
      test_images.append(t)

    test_images = np.array(test_images)
    test_images = np.reshape(test_images, (-1, 299, 299, 3))
    
    input_name = "import/" + input_layer
    output_name = "import/" + output_layer
    
    input_operation = graph.get_operation_by_name(input_name)
    output_operation = graph.get_operation_by_name(output_name)

    with tf.Session(graph=graph) as sess:                       #sess.run 開始進行預測的運算 
      results = sess.run(output_operation.outputs[0], {
        input_operation.outputs[0]: test_images
      })

    # results = np.array(results)
    # print(results)
    # print(results.shape)
    for i,c in enumerate(file_list):
      a = []
      a.append(c)
      if results[i][1]>results[i][0]:
        a.append('ok')
        a.append(results[i][1])
        a.append(results[i][0])
      else:
        a.append('ng')
        a.append(results[i][1])
        a.append(results[i][0])
      x.append(a)
   # print(x)




  with open('C:/tmp/fhi_predict1.csv','w', newline='') as scorecsv:
    score = csv.writer(scorecsv)
    score.writerow(['id','class','ok','ng'])
    for row in x:
    #print(row)
      score.writerow(row)
  scorecsv.close()

    # results = np.squeeze(results)


    # # top_k = results.argsort()[-5:][::-1]
    # labels = load_labels(label_file)
    
    # path_name = image_dir #"test"
    # bottleneck_string2=""    
    # max_x=""
    # max_y=0
    # b=1
    # for i in top_k:                                            #把預測圖片資料夾裡每張圖片預測的結果存起來
    #   # print("--b=",labels[i], results[i]) 
    #   b +=1
    #   if results[i]>max_y:
    #         max_x=labels[i]
    #         max_y=results[i]  
    #   bottleneck_string2 = bottleneck_string2+"\t"+labels[i]+": "+str(results[i])+" "  
    #   #print(labels[i], results[i])
    
    # ans2.append(file_name)
    # ans3.append(max_x)
    # ans4.append(bottleneck_string2)
    # iCount +=1
    # print('=============main for_loop end=============')
      
    # #轉出csv檔案
    # ans["Id"]= ans2
    # ans["Class"]= ans3
    # ans["Class2"]= ans4
    # ans.to_csv("C:/tmp/fhi_predict.csv", index=False)
    # print("C:\\tmp\fhi_predict.csv ",str(iCount)+" files.")
    # print('%s: ' % datetime.now())